In [53]:
import numpy as np
import os
import librosa
import keras
import pandas as pd
import time

In [2]:
import matplotlib.pyplot as plt

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [4]:
df_tp = pd.read_csv('/home/birdsong/kaggle-data/train_tp.csv')
df_fp = pd.read_csv('/home/birdsong/kaggle-data/train_fp.csv')
display(df_tp)
display(df_fp)

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70
...,...,...,...,...,...,...,...
1211,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75
1212,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25
1213,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00
1214,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20


,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max
0,00204008d,21,1,13.8400,3281.2500,14.9333,4125.0000
1,00204008d,8,1,24.4960,3750.0000,28.6187,5531.2500
2,00204008d,4,1,15.0027,2343.7500,16.8587,4218.7500
3,003b04435,22,1,43.2533,10687.5000,44.8587,13687.5000
4,003b04435,23,1,9.1254,7235.1562,15.2091,11283.3984
...,...,...,...,...,...,...,...
7776,ffd88cd84,14,1,3.2000,2531.2500,3.7867,5531.2500
7777,ffebe7313,1,1,35.2000,3843.7500,36.0960,5625.0000
7778,fff163132,17,4,22.5547,1312.5000,25.0880,7406.2500
7779,fff163132,14,1,37.0827,2531.2500,37.6693,5531.2500


In [5]:
len(df_tp[df_tp['songtype_id']==4])

128

In [6]:
1216 - 128

1088

In [7]:
len(df_fp[df_fp['songtype_id']==4])

900

In [8]:
(7781 - 900)/7969

0.8634709499309826

In [9]:
(7781 - 900)

6881

In [10]:
8997 - 128 - 900

7969

In [80]:
spectrograms = pd.read_csv('/home/birdsong/processed_data/spectrograms/spectrograms.csv').values
labels = pd.read_csv('/home/birdsong/processed_data/spectrograms/labels.csv').values[:,1]
probabilities = pd.read_csv('/home/birdsong/processed_data/spectrograms/probabilities.csv').values

In [81]:
spec = spectrograms[0]


In [82]:
spectrograms.shape

(8997, 1409)

In [13]:
labels.shape

(7969,)

In [14]:
probabilities.shape

(7969, 25)

In [15]:
1216 +7781

8997

In [84]:
arr0 = np.zeros(6881, dtype=int)
arr1 = np.ones(1088, dtype=int)
arr = np.concatenate((arr1, arr0)).reshape(-1,1)
print(arr.shape)
spec_tpfp = np.concatenate((spectrograms, arr), axis=1)

(7969, 1)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 8997 and the array at index 1 has size 7969

In [17]:
print(spec_tpfp.shape)

(7969, 4162)


In [18]:
spec_tpfp[:,:-1].shape

(7969, 4161)

In [19]:
spec_tpfp[:,-1].shape

(7969,)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(spec_tpfp[:,:-1], 
                                                    labels,
                                                    shuffle=True,
                                                    test_size = 0.15, 
                                                    random_state = 431,
                                                    stratify = spec_tpfp[:,-1])

#### Need to stratify wrt distrib of TP vs FP

In [21]:
labels

array([14, 23, 12, ..., -1, -1, -1])

In [22]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
reg = LogisticRegression(max_iter=1000).fit(X_train_scaled, y_train)

X_test_scaled = scaler.transform(X_test)

/home/birdsong/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
len(y_train)

6773

In [24]:
y_test_pred = reg.predict(X_test_scaled)

In [25]:
correct = np.count_nonzero(y_test == y_test_pred)
all = len(y_test)
correct, all, correct/all

(1084, 1196, 0.9063545150501672)

In [27]:
y_train_pred = reg.predict(X_train_scaled)
correct = np.count_nonzero(y_train == y_train_pred)
all = len(y_train)
correct, all, correct/all

(6772, 6773, 0.9998523549387273)

### Trying out some classifiers with and without PCA preprocessing

##### KNN with PCA

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

In [29]:
## Make the kfold object here
kfold = KFold(n_splits = 5,
              shuffle=True,
              random_state= 423)

In [30]:
## Make the PCA Pipeline here
pipe = Pipeline([('scale', StandardScaler()),
                 ('pca', PCA(2))])

## Fit the PCA here
pipe.fit(X_train)

## Get the transformed training data here
## Store it in the variable fit provided below
fit = pipe.transform(X_train)

In [31]:
from matplotlib.pyplot import cm

In [74]:
ks = range(1, 11)
comps = range(2,30)
n_splits = 5
## Makes a 3-D array of to record the accuracies in
pca_accs = np.zeros((n_splits, len(comps), len(ks)))
pca_accs2 = np.zeros((n_splits, len(comps), len(ks)))

i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    ## Keeps track of the cross validation split you are on
    ## This loop can be a little long
    print("CV Split", i)
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    j = 0
    for n_comps in comps:
        ## Make the PCA pipeline here
        pca_pipe = Pipeline([('scale', StandardScaler()),
                            ('pca', PCA(n_comps))])
        
        ## Fit and then get the PCA transformed tt data here
        pca_tt = pca_pipe.fit_transform(tt)
        
        ## Get the transformed holdout data here
        pca_ho = pca_pipe.transform(ho)
        
        k = 0
        for neighbors in ks:
            knn = KNeighborsClassifier(neighbors)
            
            ## Fit on the tt data
            knn.fit(pca_tt, y_tt)

            ## Get the holdout prediction
            pred = knn.predict(pca_ho)

            pca_accs[i,j,k] = accuracy_score(y_ho, pred)
            pca_accs2[i,j,k] = accuracy_score(y_tt, knn.predict(pca_tt))
            
            k = k + 1
        j = j + 1
    i = i + 1

CV Split 0
CV Split 1
CV Split 2
CV Split 3
CV Split 4


##### The above CV analysis with KNN takes ~10 min

In [75]:
## This code will print out the best # components - k combo for you
## It also prints out the highest AVG CV Accuracy
max_index = np.unravel_index(np.argmax(np.mean(pca_accs, axis=0), axis=None), 
                                       np.mean(pca_accs, axis=0).shape)


print("The pair with the highest AVG CV Accuracy was",
         "k =", ks[max_index[1]],
         "and number of components =", np.round(comps[max_index[0]],2))
print("The highest AVG CV Accuracy was", np.max(np.mean(pca_accs, axis=0)))

print("The AVG CV training Accuracy for this pair was", np.mean(pca_accs2[:,max_index[0], max_index[1]]))

The pair with the highest AVG CV Accuracy was k = 7 and number of components = 27
The highest AVG CV Accuracy was 0.8672674649937046
The AVG CV training Accuracy for this pair was 0.8728775698700639


##### It looks like the ideal PCA dimension to reduce to was the highest we tested...

##### KNN without PCA (since it liked the higher dimension anyways)

In [76]:
ks = range(1, 30)
n_splits = 5

st = time.time()

## Makes a 3-D array of to record the accuracies in
knn_accs = np.zeros((n_splits, len(ks)))
knn_accs2 = np.zeros((n_splits, len(ks)))

i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    ## Keeps track of the cross validation split you are on
    ## This loop can be a little long
    print("CV Split", i)
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    j = 0
    for neighbors in ks:
        dimst = time.time()
        knn = KNeighborsClassifier(neighbors)

        ## Fit on the tt data
        knn.fit(tt, y_tt)

        ## Get the holdout prediction
        pred = knn.predict(ho)

        knn_accs[i,j] = accuracy_score(y_ho, pred)
        knn_accs2[i,j] = accuracy_score(y_tt, knn.predict(tt))
        
        j = j + 1
        dimtime = time.time() - dimst
        print("Num Neighbors = %s" % neighbors)
        print(dimtime)
    i = i + 1
et = time.time() - st
print(et)

CV Split 0
Num Neighbors = 1
1.4076030254364014
Num Neighbors = 2
1.307558298110962
Num Neighbors = 3
1.406339168548584
Num Neighbors = 4
1.2950439453125
Num Neighbors = 5
1.3648946285247803
Num Neighbors = 6
1.2574679851531982
Num Neighbors = 7
1.4088208675384521
Num Neighbors = 8
1.4468560218811035
Num Neighbors = 9
1.4507479667663574
Num Neighbors = 10
1.4916636943817139
Num Neighbors = 11
1.4090452194213867
Num Neighbors = 12
1.3121085166931152
Num Neighbors = 13
1.4511187076568604
Num Neighbors = 14
1.4716544151306152
Num Neighbors = 15
1.4130501747131348
Num Neighbors = 16
1.3074095249176025
Num Neighbors = 17
1.4632441997528076
Num Neighbors = 18
1.4221079349517822
Num Neighbors = 19
1.3811593055725098
Num Neighbors = 20
1.291818618774414
Num Neighbors = 21
1.3915891647338867
Num Neighbors = 22
1.3298687934875488
Num Neighbors = 23
1.464447259902954
Num Neighbors = 24
1.3389487266540527
Num Neighbors = 25
1.3565983772277832
Num Neighbors = 26
1.4059550762176514
Num Neighbors = 2

In [77]:
## This code will print out the best # components - k combo for you
## It also prints out the highest AVG CV Accuracy
max_index = np.argmax(np.mean(knn_accs, axis=0))

print(knn_accs.shape)

print("The k with the highest AVG CV Accuracy was",
         "k =", ks[max_index])
print("The highest AVG CV Accuracy was", np.max(np.mean(knn_accs, axis=0)))

print("The AVG CV training Accuracy for this pair was", np.mean(knn_accs2[:,max_index]))

(5, 29)
The k with the highest AVG CV Accuracy was k = 1
The highest AVG CV Accuracy was 0.8991589768187194
The AVG CV training Accuracy for this pair was 1.0


#### Let's try out some small NN's:

In [34]:
from sklearn.neural_network import MLPClassifier

In [35]:
mlp = MLPClassifier(hidden_layer_sizes=(5,5,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.8637123745819398

In [36]:
mlp = MLPClassifier(hidden_layer_sizes=(8,5,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.8637123745819398

In [37]:
mlp = MLPClassifier(hidden_layer_sizes=(8,8,5,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.8662207357859532

In [38]:
mlp = MLPClassifier(hidden_layer_sizes=(100,50,5,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.8637123745819398

##### Something must be weird with the MLPClassifier if the accuracy scores are all the same.

#### Random forests with CV:

In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
## note this will take a minute or so to run

max_depths = range(1, 11)
n_trees = [100]

## Make an array of zeros that will hold the cv accuracies
rf_accs = np.zeros((5, len(max_depths), len(n_trees)))


i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    ## Loop through the max_depth options
    j = 0
    for depth in max_depths:
        ## Look through the number of estimators options
        k = 0
        for n_est in n_trees:
            ## This will help you keep track of where the loop is
            print(i,j,k)
            ## Make the model object, include a random state
            rf = RandomForestClassifier(max_depth=depth,
                                        n_estimators=n_est,
                                        max_samples = int(.8*len(tt)))
            
            ## Fit the model
            rf.fit(tt, y_tt)
            
            ## predict on the holdout set
            pred = rf.predict(ho)
            
            ## Record the accuracy
            rf_accs[i,j,k] = accuracy_score(y_ho,  pred)
            k = k + 1
        j = j + 1
    i = i + 1

0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 5 0
0 6 0
0 7 0
0 8 0
0 9 0
1 0 0
1 1 0
1 2 0
1 3 0
1 4 0
1 5 0
1 6 0
1 7 0
1 8 0
1 9 0
2 0 0
2 1 0
2 2 0
2 3 0
2 4 0
2 5 0
2 6 0
2 7 0
2 8 0
2 9 0
3 0 0
3 1 0
3 2 0
3 3 0
3 4 0
3 5 0
3 6 0
3 7 0
3 8 0
3 9 0
4 0 0
4 1 0
4 2 0
4 3 0
4 4 0
4 5 0
4 6 0
4 7 0
4 8 0
4 9 0


In [41]:
## This gives you the optimal depth and number of trees
max_index = np.unravel_index(np.argmax(np.mean(rf_accs, axis=0), axis=None), 
                                       np.mean(rf_accs, axis=0).shape)


print(max_depths[max_index[0]],n_trees[max_index[1]])

9 100


In [42]:
## What is the best avg cv accuracy? (Note here we only tried it out for 100 classifiers, hence the index of 0 below)
np.mean(rf_accs, axis = 0)[max_depths[max_index[0]]-1, 0]

0.8638709958739174

#### Bayes Classifiers:

In [43]:
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [44]:
## Make a holder for the accuracies
bayes_accs = np.zeros((5,3))

i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    ## Linear Discriminant Analysis
    lda = LinearDiscriminantAnalysis()
    
    lda.fit(tt, y_tt)
    lda_pred = lda.predict(ho)
    
    ## Records the accuracies
    bayes_accs[i, 0] = accuracy_score(y_ho,
                                      lda_pred)
    
    ## Quadratic Discriminant Analysis
    qda = QuadraticDiscriminantAnalysis()
    
    qda.fit(tt,y_tt)
    
    qda_pred = qda.predict(ho)
    
    ## Records the accuracies
    bayes_accs[i, 1] = accuracy_score(y_ho,
                                      qda_pred)
    
    
    ## Gaussian Naive Bayes
    nb = GaussianNB()
    
    nb.fit(tt, y_tt)
    nb_pred = nb.predict(ho)
    
    ## Records the accuracies
    bayes_accs[i, 2] = accuracy_score(y_ho,
                                      nb_pred)
    
    i = i + 1

/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [45]:
## Print the Average CV Accuracies here
np.mean(bayes_accs, axis=0)

array([0.79019475, 0.86697117, 0.08858847])

In [46]:
bayes_accs

array([[0.79852399, 0.86642066, 0.07527675],
       [0.79557196, 0.86715867, 0.09741697],
       [0.78745387, 0.87158672, 0.08339483],
       [0.79320532, 0.86262925, 0.07237814],
       [0.77621861, 0.86706056, 0.11447563]])

#### Log Reg with PCA first:

In [49]:
import time

In [50]:
st = time.time()
dimst = time.time()

comps = range(2,30)
n_splits = 5
## Makes a 3-D array of to record the accuracies in
log_reg_pca_accs = np.zeros((n_splits, len(comps)))
log_reg_pca_accs2 = np.zeros((n_splits, len(comps)))

i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    ## Keeps track of the cross validation split you are on
    ## This loop can be a little long
    print("CV Split", i)
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    j = 0
    for n_comps in comps:
        dimst = time.time()
        ## Make the PCA pipeline here
        pca_pipe = Pipeline([('scale', StandardScaler()),
                            ('pca', PCA(n_comps))])
        
        ## Fit and then get the PCA transformed tt data here
        pca_tt = pca_pipe.fit_transform(tt)
        
        ## Get the transformed holdout data here
        pca_ho = pca_pipe.transform(ho)
        
        #Logistic Regression here:
        log_reg = LogisticRegression(max_iter=10000,
                                     penalty=None)
        
        log_reg.fit(tt, y_tt)
        
        pred = log_reg.predict((ho))
        
        log_reg_pca_accs[i,j] = accuracy_score(y_ho, pred)
        log_reg_pca_accs2[i,j] = accuracy_score(y_tt, log_reg.predict(tt))
        
        j = j + 1
        dimtime = time.time() - dimst
        print(j)
        print(dimtime)
    i = i + 1
et = time.time() - st
print(et)

CV Split 0
1
75.3173816204071
2
74.08127331733704
3
74.68946123123169
4
74.618723154068
5
75.09591746330261
6
74.65824604034424
7
74.8973925113678
8
76.45788550376892
9
76.43886756896973
10
75.61822986602783
11
74.49753785133362
12
75.70071363449097
13
75.31284999847412
14
75.03169059753418
15
73.48317122459412
16
75.00130105018616
17
77.48497009277344
18
76.40409994125366
19
74.8336353302002
20
76.09874534606934
21
75.94515895843506
22
75.8968141078949
23
75.02217864990234
24
74.13532328605652


KeyboardInterrupt: 

In [51]:
st = time.time()
dimst = time.time()

comps = range(2,4)
n_splits = 5
## Makes a 3-D array of to record the accuracies in
log_reg_pca_accs = np.zeros((n_splits, len(comps)))
log_reg_pca_accs2 = np.zeros((n_splits, len(comps)))

i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    ## Keeps track of the cross validation split you are on
    ## This loop can be a little long
    print("CV Split", i)
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    j = 0
    for n_comps in comps:
        dimst = time.time()
        ## Make the PCA pipeline here
        pca_pipe = Pipeline([('scale', StandardScaler()),
                            ('pca', PCA(n_comps))])
        
        ## Fit and then get the PCA transformed tt data here
        pca_tt = pca_pipe.fit_transform(tt)
        
        ## Get the transformed holdout data here
        pca_ho = pca_pipe.transform(ho)
        
        #Logistic Regression here:
        log_reg = LogisticRegression(max_iter=10000,
                                     penalty=None)
        
        log_reg.fit(tt, y_tt)
        
        pred = log_reg.predict((ho))
        
        log_reg_pca_accs[i,j] = accuracy_score(y_ho, pred)
        log_reg_pca_accs2[i,j] = accuracy_score(y_tt, log_reg.predict(tt))
        
        j = j + 1
        dimtime = time.time() - dimst
        print(j)
        print(dimtime)
    i = i + 1
et = time.time() - st
print(et)

CV Split 0
1
74.47110629081726
2
75.22611665725708
CV Split 1
1
28.218788862228394
2
26.66051197052002
CV Split 2
1
90.16496181488037
2
90.33793878555298
CV Split 3
1
27.94273543357849
2
27.7144615650177
CV Split 4
1
23.92130708694458
2
23.379066467285156
488.2016701698303


In [52]:
## This code will print out the best # components - k combo for you
## It also prints out the highest AVG CV Accuracy
max_index = np.unravel_index(np.argmax(np.mean(log_reg_pca_accs, axis=0), axis=None), 
                                       np.mean(log_reg_pca_accs, axis=0).shape)


print("The highest AVG CV Accuracy corresponds to  number of components =", np.round(comps[max_index[0]],2))
print("The highest AVG CV Accuracy was", np.max(np.mean(log_reg_pca_accs, axis=0)))

The highest AVG CV Accuracy corresponds to  number of components = 2
The highest AVG CV Accuracy was 0.9162856535507748
